In [ ]:
from functions import *

# article_file = "data/compassion.pdf"
# csv_name = "compassion_full.csv"
# added_lit_file = "compassion_addon.csv"

###LITERATURE
# lit_df = generate_literature_csv_from_pdf(article_file, "Author", "Title", "Publication Year", output_path=csv_name)
# if added_lit_file:
#     added_lit_df = pd.read_csv(added_lit_file, keep_default_na=False, encoding="utf-8")
#     combined_df = pd.concat([added_lit_df, lit_df], ignore_index=True) #suggest to put df with more added abstracts first
#     combined_df = remove_duplicate_rows(combined_df)
#     combined_df.to_csv("compassion_full.csv", index=False)
# df = autosearch_missing_abstracts(csv_name, csv_name)
# df = manual_entry_abstracts(csv_name, csv_name)
# data_formatted = format_texts_from_csv(csv_name, "Abstract Note", "Author", "Title", "Publication Year")
# data_formatted.to_csv("trash.csv")
# vectorstore_folder_name = generate_vectorstore(data_formatted, csv_name, max_doc_size = 4000)

###CLAIMS
# claims = extract_focal_claims(article_file, max_claims=3)
# sections = analyze_paragraphs(article_file, max_paragraphs=4)

###OPPOSITION
# relevant_docs = get_relevant_docs(presented_claim, vectorstore_folder_name)
# responses, summary_response = generate_adversarial_texts(presented_claim, relevant_docs)

In [2]:
from functions import review
review("../../data/test.pdf", "../../data/test2.pdf", lower=45, upper = 2000, left = 0, manual_abstracts=False, openai_model="gpt-3.5-turbo", max_secondary=4)


---extracting references from ../../data/test.pdf

TIP: you can use the arguments upper (~763), lower (~70), and left (~10) to trim away headers, footers, or line numbers from pdf. set verbose to True to see what was cut.
  --> 3 refs extracted

---searching for additional papers based on Abstract

https://self-compassion.org/wp-content/uploads/2019/08/Daltry2018.pdf
https://research.rug.nl/files/619574774/s12144_022_02908_3.pdf
http://essay.utwente.nl/81650/13/Pasanen_BA_BMS.pdf
Self-Compassion: Theory, Method, Research, and Intervention
Kristin D Neff
https://self-compassion.org/wp-content/uploads/2019/08/Daltry2018.pdf
https://research.rug.nl/files/619574774/s12144_022_02908_3.pdf
http://essay.utwente.nl/81650/13/Pasanen_BA_BMS.pdf
Self-Compassion: Theory, Method, Research, and Intervention
Kristin D Neff

---finding abstracts


---finding related papers



100%|██████████| 3/3 [00:12<00:00,  4.33s/it]



---finding abstracts of related papers


---extracting references from ../../data/test2.pdf

TIP: you can use the arguments upper (~763), lower (~70), and left (~10) to trim away headers, footers, or line numbers from pdf. set verbose to True to see what was cut.
  --> 4 refs extracted

---searching for additional papers based on Abstract

https://pubmed.ncbi.nlm.nih.gov/35107822/
https://www.researchsquare.com/article/rs-2699401/v1.pdf
https://www.drshanesinclair.com/blog/measuring-measures-of-compassion-what-is-the-most-valid-and-reliable-compassion-scale-in-healthcare
https://www.tandfonline.com/doi/abs/10.1080/03601277.2020.1712058
https://bmjopen.bmj.com/content/11/6/e045988
https://www.researchsquare.com/article/rs-2699401/v1.pdf
https://pubmed.ncbi.nlm.nih.gov/27866323/
https://pubmed.ncbi.nlm.nih.gov/35107822/
https://self-compassion.org/wp-content/uploads/2015/12/ScaleMindfulness.pdf
https://www.drshanesinclair.com/blog/measuring-measures-of-compassion-what-is-the-most-valid-

100%|██████████| 3/3 [00:00<00:00, 172.69it/s]



---finding abstracts of related papers


---auto-web-browsing for missing data

10 abstracts missing out of 15

e. beaumont, m. durkin, c. h. : Compassion for others, self-compassion, 
scholar found paper: True
../../data/test.pdf --> https://www.sciencedirect.com/science/article/pii/S0266613815002843
length new abstract: 398

k. bluth, michael c. mullarkey: Self-Compassion: A Potential Path to Ado
scholar found paper: True
../../data/test.pdf --> https://link.springer.com/article/10.1007/s10826-018-1125-1
length new abstract: 1809

Baljinder Sahdra, Joseph Ciarr: The Compassion Balance Understanding the
scholar found paper: True
../../data/test.pdf --> https://link.springer.com/article/10.1007/s12671-023-02187-4
length new abstract: 1825

Kauser, Keyte, Regan, Nash, Fi: Exploring Associations Between Self Comp
scholar found paper: True
../../data/test.pdf --> https://link.springer.com/article/10.1007/s10880-021-09831-y
length new abstract: 980

Peter Muris 1 , 2 and Henry Ot: Self Es

In [ ]:
import pickle
sections = analyze_paragraphs(article_file, paragraph_length=1500, max_paragraphs=15, lower = 70, upper = 763, left = 10, verbose=False)
with open('sections.pkl', 'wb') as f:
    pickle.dump(sections, f)

In [ ]:
import pickle
with open('sections.pkl', 'rb') as f:
    sections = pickle.load(f)
for i in range(len(sections)):
    if sections[i]["reviewable"]:
        relevant_docs =  get_relevant_docs(sections[i], vectorstore_folder_name)
        if len(relevant_docs) > 0:
            responses, summary = generate_adversarial_texts(sections[i]["page_content"], relevant_docs, input_type="text")
            sections[i]["opposition"] = summary
            sections[i]["responses"] = [r["response"] for r in responses]
            sections[i]["references"] = [r["metadata"] for r in responses]
        else:
            sections[i]["opposition"] = "No relevant papers found."
            sections[i]["responses"] = []
            sections[i]["references"] = []
    else:
        sections[i]["opposition"] = "AI deemed section not reviewable."
        sections[i]["responses"] = []
        sections[i]["references"] = []
with open('sections_enriched.pkl', 'wb') as f:
    pickle.dump(sections, f)

In [ ]:
import pickle
with open('sections_enriched.pkl', 'rb') as f:
    sections = pickle.load(f)
create_pdf_from_dicts(sections, "trash.pdf")
import os 
os.startfile("trash.pdf")

In [ ]:
finds = get_abstract_from_google({"query": 'paper ' + 'Limitations of subjective ratings in instructional research'}, bias = [], query_only=True)

In [1]:
from functions import *
# https://pubmed.ncbi.nlm.nih.gov/35107822/
# https://www.researchsquare.com/article/rs-2699401/v1.pdf
# https://www.drshanesinclair.com/blog/measuring-measures-of-compassion-what-is-the-most-valid-and-reliable-compassion-scale-in-healthcare
# https://www.tandfonline.com/doi/abs/10.1080/03601277.2020.1712058
# https://bmjopen.bmj.com/content/11/6/e045988
# https://www.researchsquare.com/article/rs-2699401/v1.pdf
# https://pubmed.ncbi.nlm.nih.gov/27866323/
# https://pubmed.ncbi.nlm.nih.gov/35107822/
url = ""
a = extract_abstract(url, '', '', query_only=True)
a

Evaluative Conditioning: Past, Present, and Future
Tal Moran, Yahel Nudler, Yoav Bar Anan


{'Author': 'Tal Moran, Yahel Nudler, Yoav Bar Anan',
 'Title': 'Evaluative Conditioning: Past, Present, and Future',
 'Publication Year': None,
 'Abstract Note': 'Evaluative conditioning (EC) research investigates changes in the evaluation of a stimulus after co-occurrence with an affective stimulus. To explain the motivation behind this research, this review begins with an overview of the history of EC research, followed by a summary of the state of the art with respect to three key questions. First, how should EC procedures be used to influence evaluation? We provide a guide based on evidence concerning the functional properties of EC effects. Second, how does the EC effect occur? We discuss the possible mediating cognitive processes and their automaticity. Third, are EC effects ubiquitous outside the lab? We discuss the evidence for the external validity of EC research. We conclude that the most important open questions pertain to the relevance of EC to everyday life and to the leve